In [1]:
!pip install -q gradio
!pip install -q openai

In [2]:
import gradio as gr
import openai
import json

In [3]:
with open( 'GPT_SECRET_KEY.json') as f:
    data = json.load(f)

In [4]:
openai.api_key = data["API_KEY"]

In [5]:
def chatgpt_api(input_text):
    messages=[
        {"role":"system",
        "content":"You are a helpfful assistant."}
    ]
    
    if input_text:
        messages.append(
            {"role":"user",
            "content":'Summarize this text "{}" into a short and concise Dall-e2 prompt'.format(input_text)
            }
        )
        
        chat_completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
    
    reply = chat_completion.choices[0].message.content
    return reply

In [6]:
def dall_e_api(dalle_prompt):

    dalle_response = openai.Image.create(
            prompt = dalle_prompt,
            size="512x512"
        )
    image_url = dalle_response['data'][0]['url']
    return image_url

In [7]:
def whisper_transcribe(audio):
    
    audio_file = open(audio, "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    dalle_prompt = chatgpt_api(transcript["text"])
    image_url = dall_e_api(dalle_prompt)
    return transcript["text"], image_url

In [8]:
output_1 = gr.Textbox(label="Speech to Text")
output_2 = gr.Image(label="DALL-E Image")

In [ ]:
speech_interface = gr.Interface(fn = whisper_transcribe, 
                                inputs = gr.Audio(source="microphone", type="filepath"), 
                                outputs = [output_1, output_2], 
                                title = "Generate Images using Voice")

speech_interface.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\LENOVO\anaconda3\lib\site-packages\gradio\processing_utils.py:242: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
